In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from datetime import datetime, timedelta
import os
from typing import List, Dict, Tuple
import torch
from sklearn.metrics.pairwise import cosine_similarity
from glob import glob
from collections import defaultdict

/Users/lilianahotsko/Desktop/tartu_4_1/transformers/CryptoBubbles-NAACL-main/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class TemporalBubbleDetector:
    def __init__(self, price_data_path: str, tweets_path: str, model_name: str = "all-MiniLM-L6-v2",
                 sequence_window: int = 3):

        self.price_data_path = price_data_path
        self.tweets_path = tweets_path
        self.model = SentenceTransformer(model_name)
        self.sequence_window = sequence_window
        self.cached_embeddings = {}
        self.historical_data = self._load_all_historical_data()
        self.temporal_patterns = self._create_temporal_patterns()
        
    def _load_all_historical_data(self) -> Dict:
        historical_data = {}
        
        for file_path in glob(os.path.join(self.price_data_path, "*.csv")):
            symbol = os.path.basename(file_path).replace('.csv', '')
            df = pd.read_csv(file_path)
            df['datetime'] = pd.to_datetime(df['datetime'])
            historical_data[symbol] = df
            
        return historical_data
    
    def _create_temporal_patterns(self) -> Dict:
        patterns = defaultdict(list)
        
        for symbol, df in self.historical_data.items():
            df = df.sort_values('datetime')
        
            for i in range(len(df) - self.sequence_window + 1):
                window = df.iloc[i:i + self.sequence_window]
  
                pattern_key = f"{symbol}_{window.iloc[0]['datetime'].strftime('%Y-%m-%d')}"
                patterns[pattern_key] = {
                    'dates': window['datetime'].dt.strftime('%Y-%m-%d').tolist(),
                    'labels': window['label'].tolist(),
                    'symbol': symbol
                }
                
        return patterns
    
    def _process_tweets(self, tweets: List[str]) -> np.ndarray:
        if not tweets:
            return None
            
        processed_tweets = []
        for tweet in tweets:
            if isinstance(tweet, str):
                tweet = ' '.join(word for word in tweet.split() if not word.startswith('http'))
                tweet = tweet.strip()
                if tweet:
                    processed_tweets.append(tweet)
                    
        if not processed_tweets:
            return None
        
        combined_text = " ".join(processed_tweets)
        return self.model.encode([combined_text], convert_to_tensor=True)
    
    def _get_historical_tweets(self, symbol: str, date: str) -> List[str]:
        try:
            file_path = os.path.join(self.tweets_path, symbol, f"{date}.csv")
            if os.path.exists(file_path):
                df = pd.read_csv(file_path, encoding='utf-8')
                return df[df['tweet'].notna()]['tweet'].tolist()
        except Exception as e:
            print(f"Error loading tweets for {symbol} on {date}: {str(e)}")
        return []
    
    def _calculate_sequence_similarity(self, 
                                    query_sequence: Dict[str, List[str]], 
                                    pattern: Dict) -> Tuple[float, List[float]]:
        query_dates = sorted(query_sequence.keys())
        pattern_dates = pattern['dates']
        
        if len(query_dates) != len(pattern_dates):
            return 0.0, [0.0] * len(query_dates)
            
        daily_similarities = []
        
        for query_date, pattern_date in zip(query_dates, pattern_dates):
            query_embedding = self._process_tweets(query_sequence[query_date])
            
            cache_key = f"{pattern['symbol']}_{pattern_date}"
            if cache_key not in self.cached_embeddings:
                historical_tweets = self._get_historical_tweets(pattern['symbol'], pattern_date)
                historical_embedding = self._process_tweets(historical_tweets)
                if historical_embedding is not None:
                    self.cached_embeddings[cache_key] = historical_embedding
                else:
                    daily_similarities.append(0.0)
                    continue
                    
            historical_embedding = self.cached_embeddings[cache_key]
            
            if query_embedding is not None and historical_embedding is not None:
                similarity = cosine_similarity(
                    query_embedding.cpu().numpy(),
                    historical_embedding.cpu().numpy()
                )[0][0]
                daily_similarities.append(float(similarity))
            else:
                daily_similarities.append(0.0)
                
        sequence_similarity = np.mean(daily_similarities) if daily_similarities else 0.0
        return sequence_similarity, daily_similarities
    
    def predict_bubble_sequence(self, 
                              tweet_sequence: Dict[str, List[str]], 
                              top_k: int = 5) -> Dict:

        if len(tweet_sequence) != self.sequence_window:
            return {
                'error': f'Sequence must contain exactly {self.sequence_window} days of data'
            }
            
        similar_patterns = []
    
        for pattern_key, pattern in self.temporal_patterns.items():
            sequence_similarity, daily_similarities = self._calculate_sequence_similarity(
                tweet_sequence, pattern
            )
            
            if sequence_similarity > 0:
                similar_patterns.append({
                    'pattern_key': pattern_key,
                    'symbol': pattern['symbol'],
                    'dates': pattern['dates'],
                    'labels': pattern['labels'],
                    'sequence_similarity': sequence_similarity,
                    'daily_similarities': daily_similarities
                })
                

        similar_patterns.sort(key=lambda x: x['sequence_similarity'], reverse=True)
        similar_patterns = similar_patterns[:top_k]
        
        if not similar_patterns:
            return {
                'error': 'No similar patterns found',
                'predictions': []
            }
            
        query_dates = sorted(tweet_sequence.keys())
        daily_predictions = []
        
        for day_idx in range(len(query_dates)):
            day_weights = []
            day_labels = []
            
            for pattern in similar_patterns:
                weight = pattern['daily_similarities'][day_idx]
                label = pattern['labels'][day_idx]
                day_weights.append(weight)
                day_labels.append(label)
                
            if sum(day_weights) > 0:
                bubble_probability = np.average(day_labels, weights=day_weights)
            else:
                bubble_probability = 0.0
                
            daily_predictions.append({
                'date': query_dates[day_idx],
                'bubble_probability': float(bubble_probability),
                'is_bubble': bubble_probability > 0.5
            })
            
        return {
            'daily_predictions': daily_predictions,
            'similar_patterns': [
                {
                    'symbol': p['symbol'],
                    'dates': p['dates'],
                    'labels': p['labels'],
                    'similarity': float(p['sequence_similarity']),
                    'daily_similarities': [float(s) for s in p['daily_similarities']]
                }
                for p in similar_patterns
            ],
            'overall_confidence': float(similar_patterns[0]['sequence_similarity']) if similar_patterns else 0.0
        }


In [ ]:

def example_usage():
    detector = TemporalBubbleDetector(
        price_data_path="small_data/price_data",
        tweets_path="small_data/tweets",
        sequence_window=3
    )
    
    query_sequence = {
        "2024-01-01": [
            "Bitcoin hitting new highs! 🚀",
            "Mass adoption incoming!"
        ],
        "2024-01-02": [
            "Price keeps climbing, no end in sight!",
            "Everyone's talking about crypto"
        ],
        "2024-01-03": [
            "Some warning signs in the market",
            "Still bullish but cautious"
        ]
    }
    
    result = detector.predict_bubble_sequence(query_sequence)
    return result


result = example_usage()

In [6]:
result

{'daily_predictions': [{'date': '2024-01-01',
   'bubble_probability': 0.0,
   'is_bubble': np.False_},
  {'date': '2024-01-02', 'bubble_probability': 0.0, 'is_bubble': np.False_},
  {'date': '2024-01-03', 'bubble_probability': 0.0, 'is_bubble': np.False_}],
 'similar_patterns': [{'symbol': 'ABT',
   'dates': ['2018-10-18', '2018-10-19', '2018-10-20'],
   'labels': [0.0, 0.0, 0.0],
   'similarity': 0.4368457595507304,
   'daily_similarities': [0.3873489499092102,
    0.4759049415588379,
    0.44728338718414307]},
  {'symbol': 'ABT',
   'dates': ['2018-09-13', '2018-09-14', '2018-09-15'],
   'labels': [0.0, 0.0, 0.0],
   'similarity': 0.4269466797510783,
   'daily_similarities': [0.35089048743247986,
    0.48453769087791443,
    0.4454118609428406]},
  {'symbol': 'ABT',
   'dates': ['2019-07-31', '2019-08-01', '2019-08-02'],
   'labels': [0.0, 0.0, 0.0],
   'similarity': 0.4240749478340149,
   'daily_similarities': [0.39896905422210693,
    0.41189277172088623,
    0.4613630175590515]},

In [2]:
import numpy as np

example of input to llm with small data from matched embeddings

In [3]:
def create_llm_input(result):
    daily_predictions = result['daily_predictions']
    similar_patterns = result['similar_patterns']
    overall_confidence = result['overall_confidence']
    
    prompt = (
        "You are a financial market expert. Below are the results of a cryptocurrency bubble prediction model:\n\n"
        "Predictions for the next 3 days:\n"
    )
    
    for prediction in daily_predictions:
        date = prediction['date']
        bubble_probability = prediction['bubble_probability']
        is_bubble = prediction['is_bubble']
        prompt += f"Date: {date}, Bubble Probability: {bubble_probability}, Is Bubble: {is_bubble}\n"
    
    prompt += "\nSimilar patterns found in the historical data:\n"
    
    for pattern in similar_patterns:
        symbol = pattern['symbol']
        dates = ", ".join(pattern['dates'])
        labels = ", ".join(str(label) for label in pattern['labels'])
        similarity = pattern['similarity']
        daily_similarities = ", ".join(f"{sim:.2f}" for sim in pattern['daily_similarities'])
        prompt += (f"Symbol: {symbol}, Dates: {dates}, Labels: {labels}, "
                   f"Similarity: {similarity:.2f}, Daily Similarities: {daily_similarities}\n")
    
    prompt += f"\nOverall Confidence: {overall_confidence:.2f}\n\n"
    prompt += "Please analyze the results, identify any patterns or insights, and provide an interpretation of the predictions."
    
    return prompt

result = {
    'daily_predictions': [{'date': '2024-01-01', 'bubble_probability': 0.0, 'is_bubble': np.False_},
                          {'date': '2024-01-02', 'bubble_probability': 0.0, 'is_bubble': np.False_},
                          {'date': '2024-01-03', 'bubble_probability': 0.0, 'is_bubble': np.False_}],
    'similar_patterns': [{'symbol': 'ABT', 'dates': ['2018-10-18', '2018-10-19', '2018-10-20'],
                          'labels': [0.0, 0.0, 0.0], 'similarity': 0.4368457595507304,
                          'daily_similarities': [0.3873489499092102, 0.4759049415588379, 0.44728338718414307]}],
    'overall_confidence': 0.4368457595507304
}

llm_input = create_llm_input(result)

print(llm_input)


You are a financial market expert. Below are the results of a cryptocurrency bubble prediction model:

Predictions for the next 3 days:
Date: 2024-01-01, Bubble Probability: 0.0, Is Bubble: False
Date: 2024-01-02, Bubble Probability: 0.0, Is Bubble: False
Date: 2024-01-03, Bubble Probability: 0.0, Is Bubble: False

Similar patterns found in the historical data:
Symbol: ABT, Dates: 2018-10-18, 2018-10-19, 2018-10-20, Labels: 0.0, 0.0, 0.0, Similarity: 0.44, Daily Similarities: 0.39, 0.48, 0.45

Overall Confidence: 0.44

Please analyze the results, identify any patterns or insights, and provide an interpretation of the predictions.
